In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.io
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
%matplotlib inline


In [ ]:
import scipy.io

EEG = scipy.io.loadmat('/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0284/0284_008_010_EEG.mat')
EEG = EEG['val']

In [ ]:
EEG = EEG.astype(float)
EEG.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

EEG = scaler.fit_transform(EEG)

EEG

In [ ]:
EEG = np.mean(EEG,axis=0)
EEG.shape

In [ ]:
Fs = 500
fft_result = np.fft.fft(EEG)
freq = np.fft.fftfreq(len(EEG), d=1/Fs)
power_spectrum = np.abs(fft_result)
plt.plot(freq[:len(freq)//2], power_spectrum[:len(power_spectrum)//2])
plt.title("EEG Power Spectrum")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
#plt.ylim(0,1)
#plt.xlim(0,5)
plt.show()


In [ ]:
from scipy.signal import welch

f1, psd1 = welch(EEG, fs=Fs, nperseg=1024)
plt.semilogy(f1, psd1)
plt.title("Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (V²/Hz)")
plt.xlim(0,25)
plt.show()

In [ ]:
px.line(x=f,y=psd1,log_y=True)

In [ ]:
EEG2 = scipy.io.loadmat('/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0360/0360_022_060_EEG.mat')
EEG2 = EEG2['val']
EEG2 = EEG2.astype(float)
EEG2 = scaler.fit_transform(EEG2)
EEG2 = np.mean(EEG2,axis=0)

In [ ]:
fft_result = np.fft.fft(EEG2)
freq = np.fft.fftfreq(len(EEG2), d=1/Fs)
power_spectrum = np.abs(fft_result)
plt.plot(freq[:len(freq)//2], power_spectrum[:len(power_spectrum)//2])
plt.title("EEG Power Spectrum")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.yscale('log')
plt.show()

In [ ]:
f2, psd2 = welch(EEG2, fs=Fs, nperseg=1024)
plt.semilogy(f2, psd2)
plt.title("Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (V²/Hz)")
plt.xlim(0,25)

plt.show()

In [ ]:
px.line(x=f2,y=psd2,log_y=True)

In [ ]:
EEG3 = scipy.io.loadmat('/home/mario/code/SeanDominique/will-they-wake-up/data/physionet.org/files/i-care/2.1/training/0284/0284_004_007_EEG.mat')
EEG3 = EEG3['val'][0]
EEG3 = EEG3.astype(float)

In [ ]:
f, psd = welch(EEG3, fs=Fs, nperseg=2000)
plt.semilogy(f, psd)
plt.title("Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (V²/Hz)")
plt.show()

In [ ]:
px.line(x=f,y=psd,log_y=False)


In [ ]:
fft_result = np.fft.fft(EEG3)
freq = np.fft.fftfreq(len(EEG3), d=1/Fs)
power_spectrum = np.abs(fft_result)
plt.plot(freq[:len(freq)//2], power_spectrum[:len(power_spectrum)//2])
plt.title("EEG Power Spectrum")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.show()

In [ ]:
!pip install mne


In [ ]:
import mne

In [ ]:
!pip install pymatreader

In [ ]:
EEG4 = scipy.io.loadmat('/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0284/0284_085_074_EEG.mat')
EEG4 = EEG4['val']
EEG4 = EEG4.astype(float)
EEG4 = scaler.fit_transform(EEG4)
EEG4 = np.mean(EEG4,axis=0)

In [ ]:
f, psd = welch(EEG4, fs=Fs, nperseg=1024)

fig, ax = plt.subplots()
plt.semilogy(f, psd)
plt.title("Power Spectral Density")
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (V²/Hz)")
plt.xlim(0,25)
plt.show()

In [ ]:
px.line(x=f,y=[psd,psd2,psd1],log_y=False,)


In [ ]:
!ls /home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0284/*_EEG.mat

In [ ]:
path= '/home/mariorocha/code/SeanDominique/will-they-wake-up/data/raw/physionet.org/files/i-care/2.1/training/0284/'

import os
scaler = RobustScaler()
def get_eeg_files(directory):
    eeg_files = []

    for root, _, files in os.walk(directory):

        for file in files:

            if file.endswith("EEG.mat"):

                absolute_path = os.path.abspath(os.path.join(root, file))
                eeg_files.append(absolute_path)

    return sorted(eeg_files)


EEG_file_list = get_eeg_files(path)

EEG_list = []
for file_path in EEG_file_list[:-2]:

    eeg = scipy.io.loadmat(file_path)
    eeg = eeg['val']
    eeg = eeg.astype(float)
    for line in eeg:
        temp_line = line.reshape(1, -1)
        temp_line = scaler.fit_transform(temp_line)
        line = temp_line.reshape(-1)
    eeg = np.mean(eeg,axis=0)
    EEG_list.append(eeg)

EEG_list


In [ ]:
psds = []
for eeg in EEG_list:
    f, temp_psd = welch(eeg, fs=Fs, nperseg=1024)
    psds.append(temp_psd)

psds

In [ ]:
#px.line(x=f,y=psds,log_y=True)

In [ ]:
psds

In [ ]:
fig = go.Figure()

index = 0
for psd in psds:
    color_intensity = min((1 - index/len(psds) ),1) # This is to change the color of each line. Plotly doesn't like to do it, so I had to force it with an f string and rgb values.
    fig.add_trace(go.Scatter(x = f, y = psd,
        mode='lines',

        line=dict( color= f"rgb({255*color_intensity},0,{255-(255*color_intensity)})"),

        opacity = 0.5,

    ))
    index += 1

fig.update_yaxes(type='log')
fig.show()
print(color_intensity)

In [ ]:
psds = np.array(psds)
psds.shape

In [ ]:
ini_psds = psds[:,:60]

ini_psds = ini_psds.T/(ini_psds[:,0])
ini_psds.shape

In [ ]:
px.line(ini_psds.T)

In [ ]:
ini_psds.T.shape

In [ ]:
EEG_file_list[39]